In [1]:
import sys

sys.path.append("../python/")

In [2]:
%load_ext autoreload
%autoreload 2
import dask
import dask.distributed
from dask_util import DaskClient
import dask_util
import numpy as np

In [3]:
sys.path.append("/home/users/arustam/parquetvector/")

In [3]:
local_params = {
    "n_workers":4, 
    "processes" : True
    
}

import os
cluster = {
    "nanny" : True,
    "cores" : 24,
    "processes" : 1,
    "memory" : "32GB",
    "shebang" : '#!/usr/bin/env bash',
    "queue" : "serc",
    "walltime" : "24:00:00",
    "local_directory" : f'{os.environ["SCRATCH"]}/dask_local_dir',
    "death_timeout" : "15s",
    "interface" : "ib0",
    "log_directory" : f'{os.environ["SCRATCH"]}/dask_jobqueue_logs',
    "python" : "singularity run /home/users/arustam/singularity/wem_density.sif python3",
    "scheduler_options" : {
#         "idle_timeout" : "1500s",
        "worker_ttl" : "15000s"
    },
#     "worker_extra_args" : [
#         "--lifetime", "50m", 
#         "--lifetime-stagger", "9m"
#     ],
#     "job_extra_directives" : [
#         "--constraint=NO_GPU"
#     ]
}


client = DaskClient(slurm_params=cluster, n_jobs=10)

In [3]:
python = "singularity run --bind ~/repository.dev:/opt/repo,/home/users/arustam/python-solver:/opt/python-solver /home/users/arustam/singularity/wem_density.sif python3"

In [4]:
local_params = {
    "n_workers":4, 
    "processes" : True
    
}

import os
cluster = {
    "nanny" : True,
    "cores" : 24,
    "processes" : 1,
    "memory" : "16GB",
    "shebang" : '#!/usr/bin/env bash',
    "queue" : "serc",
    "walltime" : "24:00:00",
    "local_directory" : f'{os.environ["SCRATCH"]}/dask_local_dir',
    "death_timeout" : "15s",
    "interface" : "ib0",
    "log_directory" : f'{os.environ["SCRATCH"]}/dask_jobqueue_logs',
    "python" : python,
    # "worker_extra_args" : ["--lifetime", "55m", "--lifetime-stagger", "4m"],
}


client = DaskClient(slurm_params=cluster, n_jobs=10)

In [3]:
import os
small_cluster = {
    "job_name" : "copy",
    "nanny" : True,
    "cores" : 24,
    "processes" : 1,
    "memory" : "32GB",
    "shebang" : '#!/usr/bin/env bash',
    "queue" : "serc",
    "walltime" : "1:00:00",
    "local_directory" : f'{os.environ["SCRATCH"]}/dask_local_dir',
    "death_timeout" : "15s",
    "interface" : "ib0",
    "log_directory" : f'{os.environ["SCRATCH"]}/dask_copy_logs',
    "python" : "singularity run --bind ~/parquetvector:/opt/SEP/lib/python3.8 /home/groups/biondo/arustam/singularity/parquet_vector.sif python3",
    "scheduler_options" : {
        "host" : "10.19.9.27",
        "port" : "8686"
    }
}


copy_client = DaskClient(slurm_params=small_cluster, n_jobs=2)

/home/users/arustam/miniconda3/envs/dask3.10/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45276 instead
  warnings.warn(
2024-11-27 17:54:02,771 - distributed.scheduler - ERROR - Error during deserialization of the task graph. This frequently
occurs if the Scheduler and Client have different environments.
For more information, see
https://docs.dask.org/en/stable/deployment-considerations.html#consistent-software-environments



In [4]:
copy_client.getClient().scheduler

<pooled rpc to 'tcp://10.19.9.27:8686'>

In [6]:
# client.getClient().amm.stop()
copy_client.getClient().amm.start()
import gc
copy_client.getClient().run(gc.collect)  # collect garbage on all workers


{'tcp://10.51.4.42:40484': 56,
 'tcp://10.51.4.48:34718': 56,
 'tcp://10.51.5.20:46151': 56,
 'tcp://10.51.8.57:33403': 56,
 'tcp://10.51.9.70:45342': 56}

In [4]:
dask_util.save(copy_client, '/home/groups/biondo/arustam/marimba/client.dask')

NameError: name 'copy_client' is not defined

In [5]:
# dask_util.save(client, '/home/users/arustam/synth/medical/carlLine/Par/client.dask')
dask_util.save(client, '/home/users/arustam/synth/marmousi/Par/client.dask')

In [5]:
copy_client.getClient().cluster.get_logs()

NameError: name 'copy_client' is not defined

In [5]:
client.getClient().restart()

<Client: 'tcp://10.51.14.23:33793' processes=4 threads=96, memory=119.20 GiB>

In [ ]:
client.getClient().shutdown()

In [ ]:
## %load_ext autoreload
%autoreload 2
import SepVector
from __pyDaskVector import DaskVector
from __pyDaskOperator import DaskOperator
import Hypercube
import pyOperator as Op


In [ ]:

ns = [10,40,1000]
os = [0,0,0]
ds = [1,1,1]
chunks = (1,1,3)

ax = Hypercube.axis(n=1, o=1, d=1)
hyp = Hypercube.hypercube(ns=ns, ds=ds, os=os)
vec = SepVector.getSepVector(ns=ns, ds=ds, os=os)
vec.set(1)

In [ ]:
# option 1
# creating from scratch
data = DaskVector(client, vecCls=SepVector.floatVector, ns=ns, ds=ds, os=os, chunks=chunks)

In [ ]:
data.set(2)

In [ ]:
data.scale(1)

In [ ]:
data[:]

In [ ]:
# option 2
# creating from existing in-memory SepVector
daskVec = DaskVector(client, from_vector=vec, chunks=chunks)

In [ ]:
d1 = data.clone()
d2 = data.cloneSpace()

In [ ]:
client.getClient().has_what()

In [ ]:
data.getChunkHyper()

In [ ]:
daskVec.getChunkHyper()

In [ ]:
daskVec.dot(daskVec)

In [ ]:
daskVec.checkSame(daskVec)

In [ ]:
daskVec.isDifferent(daskVec)

In [ ]:
d1.isDifferent(d1)

In [ ]:
d1.isDifferent(daskVec)

In [ ]:
d2.isDifferent(d2)

In [ ]:
d2.get_workers()

In [ ]:
data.get_workers()

In [ ]:
daskVec.getHyper()

In [ ]:
scaleOp = DaskOperator(client, Op.scalingOp, daskVec, data, 4)

In [ ]:
daskVec.set(1)
for i in range(1):
    print(i)
    scaleOp.forward(False, daskVec, data)

In [ ]:
daskVec.get_workers()

In [ ]:
data.get_workers()

In [ ]:
data[:]

In [ ]:
data.min()

In [ ]:
data.max()

In [ ]:
data.set(1)
for i in range(1):
    print(i)
    scaleOp.adjoint(True, daskVec, data)

In [ ]:
daskVec[:]

In [ ]:
scaleOp.range.isDifferent(daskVec)

In [ ]:
scaleOp.range.checkSame(daskVec)

In [ ]:
nlOp = Op.NonLinearOperator(scaleOp, scaleOp)

In [ ]:
for _ in range(10):
    nlOp.set_background(daskVec)

In [ ]:
data.zero()
for i in range(10):
    data.scaleAdd(daskVec, sc2=i)
    nlOp.set_background(data)

In [ ]:
data[:]

In [ ]:
data.set(1)
for i in range(10):
    print(i)
    nlOp.nl_op.forward(True, daskVec, data)

In [ ]:
data.set(1)
for i in range(10):
    print(i)
    nlOp.nl_op.adjoint(True, daskVec, data)

In [ ]:
data.set(1)
for i in range(10):
    print(i)
    nlOp.lin_op.forward(True, daskVec, data)

In [ ]:
data.set(1)
for i in range(10):
    print(i)
    nlOp.lin_op.adjoint(True, daskVec, data)

In [ ]:
import numpy as np
import SepVector
import Hypercube
import genericIO
import WEM
import Operator

import pyVector as Vec
import pyOperator as Op
import pyNonLinearSolver as Solver
import pyLinearSolver as LinearSolver
import pyProblem as Prblm
import pyStopper as Stopper
from sys_util import logger
import json
import sys

import dask_util
from __pyDaskVector import DaskVector
from __pyDaskOperator import DaskOperator


model_pre = data.clone()
model_pre.zero()
nlOp = Op.NonLinearOperator(scaleOp,scaleOp)

#Create L2-norm linear problem
niter = 100
minb = model_pre.clone()
maxb = model_pre.clone()
minb.set(0)
maxb.set(2)

L2Prob = Prblm.ProblemL2NonLinear(model_pre,data,nlOp,minBound=minb, maxBound=maxb)
# # # #Create stopper
Stop  = Stopper.BasicStopper(niter=niter)
# # # #Create solver
# # # log = par['non_linear_solver']['prefix'].split('/')
LBFGSsolver = Solver.LBFGSBsolver(Stop, logger=logger('test.txt'))
# # # #Running the solver
# # # LBFGSsolver.setDefaults(**par['non_linear_solver'])
LBFGSsolver.run(L2Prob,verbose=True,restart=False)


In [ ]:
client.getClient().who_has()

In [ ]:
model_pre.min()

In [ ]:
model_pre.max()

In [ ]:
client.getClient().gather(scaleOp.fut)

In [ ]:
import pickle 

d = pickle.dumps(data)

In [ ]:
dd = pickle.loads(d)

In [ ]:
dd.__dict__

In [ ]:
data.__dict__

In [ ]:
dd.scaleAdd(data)

In [ ]:
f = client.getClient().submit(scaleOp.getDomain, scaleOp)

In [ ]:
f